#Python / Pandas 
Você recebeu um dataframe df com as colunas: 
- cliente_id; 
- idade; 
- genero; 
- valor_compra; 
- data_compra.

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Definir número de amostras
n = 500

# Gerar dados sintéticos
np.random.seed(42)
cliente_id = np.random.randint(1, 20, size=n)
idade = np.random.triangular(left=18, mode=25, right=75, size=n).astype(int)
genero = np.random.choice(['Masculino', 'Feminino', 'Outro'], size=n, p=[0.25, 0.70, 0.05])
valor_compra = np.round(np.random.normal(300, 250, size=n), 2)
data_compra = pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2025-09-01"), size=n))

# Criar DataFrame
df = pd.DataFrame({
    'cliente_id': cliente_id,
    'idade': idade,
    'genero': genero,
    'valor_compra': valor_compra,
    'data_compra': data_compra
})

# Consistência
df = df.loc[df['valor_compra'] > 0]

# Formatar datas
df['data_compra'] = pd.to_datetime(df['data_compra']).dt.strftime('%Y-%m-%d')

# Apresentar dataframe
display(df)


In [0]:
# Plotar histograma
plt.figure(figsize=(4.2, 2.4))
plt.hist(df['idade'], bins=30, color='gray', alpha=0.75)
plt.xlabel('Idade')
plt.ylabel('Frequência')
plt.xticks(np.arange(15, 75, 5))
plt.show()

In [0]:
# Contar número de clientes por gênero
contagem_genero = df['genero'].value_counts()
plt.figure(figsize=(4.2, 2.4))
contagem_genero.plot(kind='bar', color='gray', alpha=0.75)
plt.xlabel('Gênero')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [0]:
# Plotar histograma
plt.figure(figsize=(4.2, 2.4))
plt.hist(df['valor_compra'], bins=30, color='gray', alpha=0.75)
plt.xlabel('Compra (R$)')
plt.ylabel('Frequência')
plt.show()

## Questões: 
1. Escreva um código em Python usando a biblioteca pandas que calcule o valor total gasto por cliente.

2. Escreva um código em Python usando a biblioteca pandas que filtre apenas 
os clientes com idade entre 25 e 40 anos e compras acima de 500 reais. 

3. Escreva um código em Python que cria um gráfico de barras mostrando o 
valor total de compras por faixa etária (18-25, 26-35, 36-50, 50+)

4. Descreva os passos e escreva um código em Python de como realizaria uma análise exploratória para investigar se existe correlação entre o valor total de compras e a idade dos clientes ao longo dos meses. Elabore hipóteses envolvendo múltiplas variáveis, como gênero, faixa etária e valor das compras, e utilize gráficos apropriados (por exemplo, de dispersão ou calor) para ilustrar possíveis relações entre essas variáveis. Avalie quais fatores podem estar associados ao aumento do volume de compras em determinados períodos.

In [0]:
# Agrupar dados por cliente
total_por_cliente = df.groupby('cliente_id', as_index=True)['valor_compra'].sum()
total_por_cliente = total_por_cliente.to_frame(name='valor_total_gasto').reset_index().sort_values(by='valor_total_gasto', ascending=False)
display(total_por_cliente)

In [0]:
# Filtragem dos dados
clientes_filtrados = df.loc[
    (df['idade'] >= 25) & 
    (df['idade'] <= 40) & 
    (df['valor_compra'] > 500)
]

# Ordenação dos resultados
clientes_filtrados = clientes_filtrados.sort_values(
    by='valor_compra', 
    ascending=False
)

# Apresentar o dataframe
display(clientes_filtrados)

In [0]:
import matplotlib.pyplot as plt

# Definir faixas etárias
bins = [18, 25, 35, 45, 55, 65, 70]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-70']
df['faixa_etaria'] = pd.cut(df['idade'], bins=bins, labels=labels, right=False)

# Calcular valor total de compras por faixa etária
total_por_faixa = df.groupby('faixa_etaria')['valor_compra'].sum()/1000

# Plotar gráfico de barras
plt.figure(figsize=(4.2, 2.4))
total_por_faixa.plot(kind='bar', color='gray', alpha=0.75)
plt.xlabel('Faixa Etária')
plt.ylabel('Valor Total (milhares R$)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

## Análise Exploratória:

Investigar a correlação entre o valor total de compras e a idade dos clientes ao longo dos meses, considerando também variáveis como gênero e faixa etária, buscar identificar fatores associados ao aumento do volume de compras em determinados períodos.

### Análise do Valor Total de Compras por Gênero ao Longo do Tempo
Investigar como o valor total de compras se distribui ao longo do tempo entre clientes do sexo masculino e feminino, identificando possíveis diferenças no comportamento de consumo por gênero

**Objetivos:**
- Visualização clara do comportamento de consumo mensal para cada gênero.
- Possível identificação de diferenças de gastos entre homens e mulheres.
- Permite detectar picos de consumo em determinados períodos do ano.


In [0]:
import pandas as pd
import matplotlib.pyplot as plt

# Garantir que a data esteja no formato datetime
df['data_compra'] = pd.to_datetime(df['data_compra'])

# Criar coluna mês/ano
df['mes_ano'] = df['data_compra'].dt.to_period('M')

# Agrupar por mês e gênero, somando o valor das compras
valor_por_genero = df.groupby(['mes_ano', 'genero'])['valor_compra'].sum().unstack(fill_value=0)
valor_por_genero['Total'] = valor_por_genero.sum(axis=1)

# Converter índice para string para plotagem
meses = valor_por_genero.index

# Plotar gráfico de linha
plt.figure(figsize=(7.2, 3.6))
plt.plot(valor_por_genero.index.astype(str), valor_por_genero['Masculino']/1000, marker='s', label='Masculino', alpha=0.7)
plt.plot(valor_por_genero.index.astype(str), valor_por_genero['Feminino']/1000, marker='o', label='Feminino', alpha=0.7)
plt.plot(valor_por_genero.index.astype(str), valor_por_genero['Outro']/1000, marker='.', label='Outro', alpha=0.7)
plt.plot(valor_por_genero.index.astype(str), valor_por_genero['Total']/1000, color='black', linestyle='--', marker='^', label='Total', alpha=0.7)
plt.xlabel('Mês/Ano')
plt.ylabel('Valor Total de Compras (milhares R$)')
plt.yticks(ticks=np.arange(0, 13, 2))
plt.xticks(ticks=np.arange(0, len(meses), 3), labels=meses[::3], rotation=45)
plt.legend(ncols=4)
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

print(f"Média: {valor_por_genero['Total'].mean():.2f}")
print(f"Desvio padrão: {valor_por_genero['Total'].std():.2f}")


### Custo acumulado

In [0]:


# Plotar gráfico de linha acumulado
plt.figure(figsize=(7.2, 3.6))
macador = ['s', 'o', '.', '^']
for i, col in enumerate(['Masculino', 'Feminino', 'Outro', "Total"]):
    plt.plot(valor_acumulado.index.astype(str), valor_acumulado[col]/1000, marker=macador[i], alpha=0.75, label=col)
plt.xlabel('Mês/Ano')
plt.ylabel('Valor Acumulado (milhares R$)')
plt.xticks(ticks=np.arange(0, len(meses), 3), labels=meses[::3], rotation=45)
plt.legend()
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Calcular valores acumulados
valor_acumulado = valor_por_genero.cumsum()

# Regressão linear
X = np.arange(len(valor_por_genero.index)).reshape(-1, 1)  # variável independente
y = valor_acumulado['Total'].to_numpy().reshape(-1, 1)  # variável dependente
model = LinearRegression()
model.fit(X, y)
coef = model.coef_[0]
print("Receita marginal (R$/mensal): R${:.2f}".format(coef[0]))
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
print(f"Score r2: {r2:.2f}")

# Plotar gráfico de linha acumulado
plt.figure(figsize=(7.2, 3.6))
macador = ['s', 'o', '.', '^']
for i, col in enumerate(['Masculino', 'Feminino', 'Outro', "Total"]):
    plt.plot(valor_acumulado.index.astype(str), valor_acumulado[col]/1000, marker=macador[i], alpha=0.75, label=col)
plt.plot(valor_acumulado.index.astype(str), y_pred/1000, color='black', linestyle='--', label='Tendência')
plt.xlabel('Mês/Ano')
plt.ylabel('Valor Acumulado (milhares R$)')
plt.xticks(ticks=np.arange(0, len(meses), 3), labels=meses[::3], rotation=45)
plt.legend(ncols=2)
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()